In [1]:
import numpy as np
import pandas as pd
import psycopg2
import os

In [2]:
cwd = os.getcwd()
path1 = os.path.join(cwd,r'01-05T142356_017_mode1.csv')

data = pd.read_csv(path1)
data.columns = data.columns.str.replace(':','_')
data

,timestamp,pCut__Motor_Torque,pCut__CTRL_Position_controller__Lag_error,pCut__CTRL_Position_controller__Actual_position,pCut__CTRL_Position_controller__Actual_speed,pSvolFilm__CTRL_Position_controller__Actual_position,pSvolFilm__CTRL_Position_controller__Actual_speed,pSvolFilm__CTRL_Position_controller__Lag_error,pSpintor__VAX_speed
0,0.008000,0.269432,-0.066662,669457905,-123.596191,15408586,3020.085205,0.855075,1799.999756
1,0.012000,0.274801,-0.040878,669457904,-154.495239,15408598,3030.152100,0.812925,1799.999756
2,0.016000,0.243310,-0.014990,669457903,-202.560425,15408610,3077.969971,0.756688,1799.999756
3,0.020000,0.233173,0.000410,669457902,-233.459473,15408623,3027.635498,0.796055,1799.999756
4,0.024000,0.338183,-0.024590,669457902,-271.224976,15408635,3037.702393,0.835454,1799.999756
...,...,...,...,...,...,...,...,...,...
2043,8.179999,0.433942,0.010651,669472147,-1953.506470,15444732,3742.388916,1.045140,1799.999756
2044,8.183999,0.585278,-0.024151,669472139,-1974.105835,15444747,3744.905762,1.120481,1799.999756
2045,8.187999,0.464644,0.003002,669472131,-2121.734619,15444762,3749.939209,1.176705,1799.999756
2046,8.191999,0.496963,0.010445,669472122,-2132.034180,15444777,3838.024902,1.202372,1799.999756


In [11]:
import psycopg2
from io import StringIO

# Connection parameters
param_dic = {
    "host"      : "localhost",
    "database"  : "sensordata",
    "user"      : "postgres",
    "password"  : "JNt8hSW4l+MJ]p;.g[WG"
}

#Credit to Naysan Saran. https://naysan.ca/2020/06/21/pandas-to-postgresql-using-psycopg2-copy_from/

def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn


def copy_from_stringio(conn, df, table):
    """
    Here we are going save the dataframe in memory 
    and use copy_from() to copy it to the table
    """
    # save dataframe to an in memory buffer
    buffer = StringIO()
    df.to_csv(buffer, index_label='id', header=False)
    buffer.seek(0)

    cursor = conn.cursor()
    try:
        cursor.copy_from(buffer, table, sep=",")
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:

        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("copy_from_stringio() done")
    cursor.close()

In [14]:
conn = connect(param_dic) # connect to the database
cursor = conn.cursor() # set cursor

# CREATE TABLE
list1 = list(data.keys()) #use column labels for SQL query
string1 = "CREATE TABLE industrial_component_degradation" (id int" #escape quotation marks to make it case sensitive
for label in list1:
    string1 += (", " + label + " decimal")
string1 = string1 + ");"
print(string1)
cursor.execute(string1) #execute SQL query

copy_from_stringio(conn, data, "industrial_component_degradation") # copy the dataframe to SQL
conn.close() # close the connection

Connecting to the PostgreSQL database...
Connection successful
CREATE TABLE "OneYearIndustrialComponentDegradation" (id int, timestamp decimal, pCut__Motor_Torque decimal, pCut__CTRL_Position_controller__Lag_error decimal, pCut__CTRL_Position_controller__Actual_position decimal, pCut__CTRL_Position_controller__Actual_speed decimal, pSvolFilm__CTRL_Position_controller__Actual_position decimal, pSvolFilm__CTRL_Position_controller__Actual_speed decimal, pSvolFilm__CTRL_Position_controller__Lag_error decimal, pSpintor__VAX_speed decimal);
copy_from_stringio() done
